In [15]:
from keras.applications import vgg16

In [16]:
img_rows, img_cols = 224, 224 

model = vgg16.VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

In [17]:
for layer in model.layers:
    layer.trainable = False

In [18]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [19]:
def layer_adder(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 3 (Young, Middle, Old)
num_classes = 2

FC_Head = layer_adder(model, num_classes)

model = Model(inputs = model.input, outputs = FC_Head)

print(model.summary())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [21]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = "C://Users//hp//Desktop//MLOPS//FaceRecognizationTransferLearning//Dataset//Training_dataset//"
validation_data_dir = "C://Users//hp//Desktop//MLOPS//FaceRecognizationTransferLearning//Dataset//test_dataset//"

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 1
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 140 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [22]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("face_detectiom_TL.h15",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam',
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 140
nb_validation_samples = 60

# We only train 5 EPOCHS 
epochs = 5
batch_size = 1

history = model.fit_generator(
    train_generator,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    )


Epoch 1/5
140/140 [==============================] - 107s 767ms/step - loss: 0.7017 - accuracy: 0.6571 - val_loss: 0.1575 - val_accuracy: 0.5167

Epoch 00001: val_loss improved from inf to 0.15746, saving model to face_detectiom_TL.h15
Epoch 2/5
140/140 [==============================] - 113s 804ms/step - loss: 0.3411 - accuracy: 0.8500 - val_loss: 2.2157 - val_accuracy: 0.7833

Epoch 00002: val_loss did not improve from 0.15746
Epoch 3/5
140/140 [==============================] - 111s 795ms/step - loss: 0.4649 - accuracy: 0.8786 - val_loss: 0.3812 - val_accuracy: 0.9833

Epoch 00003: val_loss did not improve from 0.15746
Epoch 4/5
140/140 [==============================] - 109s 782ms/step - loss: 0.1484 - accuracy: 0.9643 - val_loss: 0.0039 - val_accuracy: 1.0000

Epoch 00004: val_loss improved from 0.15746 to 0.00389, saving model to face_detectiom_TL.h15
Epoch 5/5
140/140 [==============================] - 110s 787ms/step - loss: 0.1273 - accuracy: 0.9286 - val_loss: 0.0926 - val_ac